# Results p. 3: Boundary Sensitivity
## Fixation Boundaries

In [1]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"

### Set Constants

In [2]:
THRESHOLD = 5       # temporal threshold for analyzing d'
METRIC = peyes.constants.D_PRIME_STR        # can also be `peyes.constants.F1_STR` or `peyes.constants.CRITERION_STR`
EVENT_NAME, EVENT_LABEL = "Fixation", 1    # EventLabelEnum.FIXATION.value

GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH
LINE_WIDTH, ERROR_WIDTH = 2 * ZERO_LINE_WIDTH, ZERO_LINE_WIDTH
MARKER_SIZE = LINE_WIDTH * 2

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=26, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=23, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_TITLE_STANDOFF = 2

## Load Data

In [3]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

## Multi-Threshold Figure
### (not in the paper)
This figure shows the `METRIC` values at increasing temporal thresholds: $$\Delta t = 0, 1, ..., 20$$
Each detector is shown in a different color, with error bars showing the SEM across recordings.
The left colomn show results when `GT1` (_RA_) is used as the ground truth, while the right column shows results when `GT2` (_MN_) is used.

In [4]:
W, H = 1300, 450

multi_thresholds_figure = ch_sdt.multi_channel_figure(
    csdt_metrics,
    metric=METRIC,
    yaxis_title=r"d'", show_other_gt=True,
    error_bars='std', show_err_bands=False,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_hspace=0, subplots_vspace=0,
)

multi_thresholds_figure.update_layout(
    width=W, height=H,
    title=dict(text=EVENT_NAME + "s", y=0.975, x=0.5, xanchor='center'),
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',

    # remove axis grids
    xaxis=dict(showgrid=False, zeroline=False, showline=False), yaxis=dict(showgrid=False, zeroline=False, showline=False),
    xaxis2=dict(showgrid=False, zeroline=False, showline=False), yaxis2=dict(showgrid=False, zeroline=False, showline=False),
    xaxis3=dict(showgrid=False, zeroline=False, showline=False), yaxis3=dict(showgrid=False, zeroline=False, showline=False),
    xaxis4=dict(showgrid=False, zeroline=False, showline=False), yaxis4=dict(showgrid=False, zeroline=False, showline=False),

    # move legend to bottom
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.05),
    # showlegend=False,   # hide legend
    margin=dict(l=10, r=0, b=10, t=0, pad=0),
)
multi_thresholds_figure.layout.annotations = []   # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 6, '', True
# save_fig(multi_thresholds_figure, FIG_ID, PANEL_ID, f"{TITLE.lower()}discrimination_multi_threshold-{METRIC.lower()}", IS_SUPP)

# multi_thresholds_figure.show()

## Statistical Analysis
### Onsets

In [5]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc = ch_sdt.post_hoc_table(
    onset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([onset_statistics, onset_pvalues, onset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  65.783226  2.984602e-12    True
                 RA  93.570652  5.484541e-18    True
d_prime          MN  69.530323  5.104016e-13    True
                 RA  90.157609  2.809965e-17    True
f1               MN  69.158710  6.082556e-13    True
                 RA  71.474359  2.037246e-13    True
false_alarm_rate MN  44.961039  4.764385e-08    True
                 RA  55.673507  3.388724e-10    True
precision        MN  49.616580  5.610951e-09    True
                 RA  68.967213  6.657793e-13    True
recall           MN  73.873016  6.547338e-14    True
                 RA  93.607843  5.387685e-18    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.    n.s.     n.s.
         RA      --    n.s.     ***      **    n.s.    n.s.     n.s.
ivvt     MN  0.9467      --     ***     ***    n.s.    n.s.     n.s.
         RA  0.9590      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.0262  0.0002      --    n.s.     ***      **     n.s.
         RA  0.0010  0.0000      --    n.s.     ***     ***     n.s.
idvt     MN  0.0322  0.0003  1.0000      --     ***      **     n.s.
         RA  0.0012  0.0000  1.0000      --     ***     ***     n.s.
engbert  MN  0.5759  0.9926  0.0000  0.0000      --    n.s.        *
         RA  0.5373  0.9841  0.0000  0.0000      --    n.s.        *
nh       MN  0.9986  0.9987  0.0032  0.0041  0.8863      --     n.s.
         RA  1.0000  0.9919  0.0002  0.0003  0.7200      --     n.s.
remodnav MN  0.9046  0.2653  0.4979  0.5397  0.0383  0.6100       --
         RA  0.8462  0.2215  0.1394  0.1546  0.0193  0.6924       --

### Offsets

In [6]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc = ch_sdt.post_hoc_table(
    offset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([offset_statistics, offset_pvalues, offset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                  
criterion        MN   68.945736  6.725602e-13    True
                 RA   98.235721  5.854802e-19    True
d_prime          MN   72.000000  1.588873e-13    True
                 RA   94.242974  3.974055e-18    True
f1               MN   58.977376  7.260057e-11    True
                 RA   79.778226  3.970301e-15    True
false_alarm_rate MN   33.683089  7.744746e-06    True
                 RA   53.316033  1.013328e-09    True
precision        MN   57.643979  1.353008e-10    True
                 RA   68.360476  8.863997e-13    True
recall           MN   73.435484  8.054783e-14    True
                 RA  102.486334  7.595625e-20    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.      **      ***
         RA      --    n.s.       *       *    n.s.     ***      ***
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.      **      ***
         RA  0.9999      --    n.s.    n.s.    n.s.      **      ***
idt      MN  0.0925  0.1755      --    n.s.       *    n.s.     n.s.
         RA  0.0275  0.0850      --    n.s.      **    n.s.     n.s.
idvt     MN  0.1333  0.2382  1.0000      --       *    n.s.     n.s.
         RA  0.0393  0.1138  1.0000      --      **    n.s.        †
engbert  MN  0.9998  0.9967  0.0285  0.0450      --     ***      ***
         RA  0.9907  0.9338  0.0012  0.0020      --     ***      ***
nh       MN  0.0023  0.0067  0.9592  0.9252  0.0004      --     n.s.
         RA  0.0004  0.0024  0.9670  0.9451  0.0000      --     n.s.
remodnav MN  0.0000  0.0000  0.2410  0.1778  0.0000  0.8634       --
         RA  0.0000  0.0000  0.0846  0.0620  0.0000  0.5875       --

## Figure: Distribution @ Threshold
### (not in the paper)
This figure shows the distribution of `METRIC` at the predefined temporal `THRESHOLD`, for each detector across recordings.
The right-side of each violin is w.r.t. `GT1` (_RA_) as the ground truth annotator, and the left-side is w.r.t. `GT2` (_MN_).

In [7]:
W, H = 600, 450

onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

######################################
## COMBINE ONSET AND OFFSET FIGURES ##
######################################

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=[f"{EVENT_NAME} {peyes.constants.ONSET_STR}", f"{EVENT_NAME} {peyes.constants.OFFSET_STR}"]
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles

# FIG_ID, IS_SUPP = 5, False
# save_fig(discriminability_figure, FIG_ID, 'left', 'fixation-discriminability', IS_SUPP)

# discriminability_figure.show()

## Final Figure
### (Shown in the paper)
- Combine Multi-Threshold line-plots with Single-Threshold violin-plots
- use only one GT annotator

In [20]:
WIDTH, HEIGHT = 1600, 900
COLUMN_TITLES = [f"{EVENT_NAME} {peyes.constants.ONSET_STR.capitalize()}s", f"{EVENT_NAME} {peyes.constants.OFFSET_STR.capitalize()}s",]


def _convert_line_traces(line_fig: go.Figure, new_fig: go.Figure, gt: str) -> go.Figure:
    if gt == "RA":
        other = "MN"
        allowed_yaxes = ["y", "y3"]
    elif gt == "MN":
        other = "RA"
        allowed_yaxes = ["y2", "y4"]
    else:
        raise ValueError("Unknown GT: {}".format(gt))

    for trace in line_fig.data:
        if not trace['yaxis'] in allowed_yaxes:
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr["showlegend"] = False
        new_tr["line_width"] = LINE_WIDTH
        new_tr["error_y"]["thickness"] = ERROR_WIDTH
        new_tr["marker"]["size"] = LINE_WIDTH * 2

        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
            new_tr["marker"]["color"] = new_tr["line"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] in ['y', 'y2']   # onset traces are in top row
        new_fig.add_trace(new_tr, row=1, col=1 if is_onset else 2)
    return new_fig


def _convert_violin_traces(violin_fig, new_fig, gt: str) -> go.Figure:
    other = "MN" if gt == "RA" else "RA"
    for trace in violin_fig.data:
        if trace['scalegroup'] != METRIC:
            # ignore non `METRIC` traces
            continue
        if not trace["name"].startswith(gt):
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr['scalegroup'] = "VIOLINS"
        new_tr["showlegend"] = False

        # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
        if new_tr["x0"] == "Other GT":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
        elif new_tr["x0"].startswith("i"):
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
        elif new_tr["x0"] == "remodnav":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
        new_tr['x'] = new_tr['y']

        # reset irrelevant props
        new_tr['y'] = new_tr['x0'] = None
        new_tr["showlegend"] = new_tr['points'] = False
        new_tr["side"] = "positive"
        new_tr["width"] = 1.8
        new_tr["box"] = dict(visible=False, width=0.95, line=dict(width=1))
        new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
        new_tr["opacity"] = 1
        new_tr["visible"] = True

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] == 'y'
        new_fig.add_trace(new_tr, row=2, col=1 if is_onset else 2)
    return new_fig


def convert_figs(line_fig, violin_fig, gt: str) -> go.Figure:
    # copy traces to new figure
    new_fig = make_subplots(
        rows=len(COLUMN_TITLES), cols=len(COLUMN_TITLES),
        shared_xaxes=False, shared_yaxes=True,
        vertical_spacing=0.12, horizontal_spacing=0.02,
        column_titles=COLUMN_TITLES,
    )
    _convert_line_traces(line_fig, new_fig, gt)
    _convert_violin_traces(violin_fig, new_fig, gt)

    # update legends
    new_fig.for_each_trace(lambda tr: tr.update(showlegend=tr['yaxis'] == 'y'))  # legend only shows one set of lines

    # update subtitles
    new_fig.for_each_annotation(lambda ann: ann.update(text=f"<b>{ann.text}</b>", font=TITLE_FONT, textangle=0, ))

    # add red rectangles in the top plots to to highlight the bottom subplots' origin
    for c in range(len(COLUMN_TITLES)):
        if gt=="RA":
            y0 = -0.65 if c == 0 else -0.45
            y1 = 4.75 if c == 0 else 5.25
        else:
            y0 = -0.65 if c == 0 else -0.1
            y1 = 3.65 if c == 0 else 5.3
        new_fig.add_shape(
            row=1, col=c + 1,
            type="rect",
            x0=THRESHOLD-0.5, x1=THRESHOLD+0.5,
            y0=y0, y1=y1,
            line_color="red", line_width=LINE_WIDTH,
        )

    # add axis labels
    for r in range(len(COLUMN_TITLES)):
        xaxis_title = "Δt (samples)" if r == 0 else r"$d'$"
        yaxis_title = r"$d'$" if r == 0 else "Detector"
        for c in range(len(COLUMN_TITLES)):
            new_fig.update_xaxes(
                row=r + 1, col=c + 1,
                title=dict(text=xaxis_title, font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=r == 0, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                tickfont=AXIS_TICK_FONT,
            )
            new_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=yaxis_title if c == 0 else "", font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                showticklabels=r==0 and c==0, tickfont=AXIS_TICK_FONT, tickangle=0 if r==0 else 30,
            )

    # add row annotations "A" and "B"
    for ann in ["A", "B"]:
        new_fig.add_annotation(
            text=f"<b>{ann}</b>", font={**TITLE_FONT, **dict(size=36)}, showarrow=False,
            xref="paper", yref="paper", xanchor="right", yanchor="top", x=0, y=1.05 if ann == "A" else 0.46
        )


    new_fig.update_layout(
        font_family=FONT_FAMILY,
        width=WIDTH, height=HEIGHT,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            orientation="h", bgcolor='rgba(0, 0, 0, 0)',
            yanchor="top", y=-0.06, xanchor="center", x=0.5,
            font=AXIS_TICK_FONT, itemwidth=90,
        ),
    )
    return new_fig

In [23]:
GT = "RA"
NAME = f"fig4-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

In [22]:
GT = "MN"
NAME = f"supp_fig_G1-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

In [11]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics
csdt_subset = csdt_metrics.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)

### Onsets

In [12]:
onset_statistics_subset, onset_pvalues_subset, onset_nemenyi_subset, onset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc_subset = ch_sdt.post_hoc_table(
    onset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [onset_statistics_subset, onset_pvalues_subset, onset_pvalues_subset <= ALPHA],
        axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  57.722892  1.304103e-10    True
                 RA  74.076923  5.944516e-14    True
d_prime          MN  59.819277  4.898027e-11    True
                 RA  73.058824  9.627196e-14    True
f1               MN  58.987952  7.224269e-11    True
                 RA  64.650544  5.084181e-12    True
false_alarm_rate MN  37.370651  1.490874e-06    True
                 RA  44.951163  4.785960e-08    True
precision        MN  40.148260  4.259535e-07    True
                 RA  54.641638  5.475727e-10    True
recall           MN  64.516229  5.415476e-12    True
                 RA  76.120650  2.255990e-14    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.    n.s.     n.s.
         RA      --    n.s.      **      **    n.s.    n.s.     n.s.
ivvt     MN  0.9719      --      **      **    n.s.    n.s.     n.s.
         RA  0.9657      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.0438  0.0010      --    n.s.     ***       *     n.s.
         RA  0.0067  0.0001      --    n.s.     ***       *     n.s.
idvt     MN  0.0545  0.0014  1.0000      --     ***       *     n.s.
         RA  0.0083  0.0001  1.0000      --     ***       *     n.s.
engbert  MN  0.7586  0.9979  0.0000  0.0001      --    n.s.     n.s.
         RA  0.6808  0.9954  0.0000  0.0000      --    n.s.     n.s.
nh       MN  1.0000  0.9898  0.0251  0.0319  0.8466      --     n.s.
         RA  1.0000  0.9365  0.0113  0.0139  0.5914      --     n.s.
remodnav MN  0.9240  0.3768  0.5640  0.6116  0.1055  0.8623       --
         RA  0.9617  0.4533  0.1701  0.1928  0.1140  0.9814       --

### Offsets

In [13]:
offset_statistics_subset, offset_pvalues_subset, offset_nemenyi_subset, offset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc_subset = ch_sdt.post_hoc_table(
    offset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [offset_statistics_subset, offset_pvalues_subset, offset_pvalues_subset <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  60.751131  3.167040e-11    True
                 RA  79.727891  4.066483e-15    True
d_prime          MN  65.692308  3.115044e-12    True
                 RA  81.741497  1.559907e-15    True
f1               MN  52.760870  1.310848e-09    True
                 RA  70.466926  3.279607e-13    True
false_alarm_rate MN  31.784284  1.794689e-05    True
                 RA  42.165899  1.705161e-07    True
precision        MN  48.872727  7.906293e-09    True
                 RA  55.673143  3.389299e-10    True
recall           MN  64.424528  5.653950e-12    True
                 RA  85.393365  2.736281e-16    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.     ***      ***
         RA      --    n.s.    n.s.    n.s.    n.s.     ***      ***
ivvt     MN  0.9995      --    n.s.    n.s.    n.s.      **      ***
         RA  0.9984      --    n.s.    n.s.    n.s.     ***      ***
idt      MN  0.1910  0.4449      --    n.s.    n.s.    n.s.     n.s.
         RA  0.1087  0.3677      --    n.s.       *    n.s.     n.s.
idvt     MN  0.2194  0.4873  1.0000      --    n.s.    n.s.     n.s.
         RA  0.1342  0.4202  1.0000      --       *    n.s.     n.s.
engbert  MN  1.0000  0.9977  0.1347  0.1571      --     ***      ***
         RA  0.9994  0.9577  0.0259  0.0342      --     ***      ***
nh       MN  0.0006  0.0048  0.7252  0.6856  0.0003      --     n.s.
         RA  0.0001  0.0010  0.5665  0.5104  0.0000      --     n.s.
remodnav MN  0.0000  0.0001  0.2312  0.2018  0.0000  0.9898       --
         RA  0.0000  0.0000  0.1100  0.0881  0.0000  0.9838       --